<a href="https://colab.research.google.com/github/JungHwan-Oh707/github-arduino/blob/main/%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#include <ESP8266WiFi.h>
#include <DHT.h>
#include <ESP8266HTTPClient.h>

#define DHTPIN A0
#define LM35PIN A1
#define SOILPIN A2
#define DHTTYPE DHT11

const char* ssid = "YourWiFiNetwork";
const char* password = "YourWiFiPassword";
const char* host = "api.openai.com";  // OpenAI GPT-3 endpoint



DHT dht(DHTPIN, DHTTYPE);



String askGPT(String message) {
  WiFiClientSecure client;
 
if (!client.connect(host, 443)) {
return "connection failed";
  }
  String url = "/v1/engines/davinci-codex/completions";
  client.println("POST " + url + " HTTP/1.1");
  client.println("Host: " + String(host));
  client.println("Content-Type: application/json");
  client.println("Authorization: Bearer YOUR_OPENAI_KEY");
  client.println("User-Agent: ESP8266");
  client.print("Content-Length: ");
  client.println(message.length());
  client.println();
  client.println(message);


while(client.connected()) {
    String line = client.readStringUntil('\n');
    
if (line == "\r") {
break;
    }
  }

  String response = client.readStringUntil('\n');
  return response;
}

void setup() {
  Serial.begin(115200);
  WiFi.begin(ssid, password);
  
 
while (WiFi.status() != WL_CONNECTED) {
    Serial.println("Connecting to WiFi...");

delay(1000);
  }
  Serial.println("Connected to WiFi");
  dht.begin();
}

void loop() {
    delay(2000);
float temperature = dht.readTemperature(); 
float humidity = dht.readHumidity();

int lm35Value = analogRead(LM35PIN);
float lm35Voltage = lm35Value * (3.3 / 1024.0);
float lm35Temperature = lm35Voltage * 100.0;

int soilMoistureValue = analogRead(SOILPIN);


if (isnan(temperature) || isnan(humidity)) {
    Serial.println("Failed to read from DHT sensor!");
    return;
  }

  Serial.print("DHT Temperature: ");
  Serial.print(temperature);
  Serial.print(" °C\t");
  Serial.print("DHT Humidity: ");
  Serial.print(humidity);
  Serial.println(" %");

  Serial.print("LM35 Temperature: ");
  Serial.print(lm35Temperature);
  Serial.println(" °C");

  Serial.print("Soil Moisture: ");
  Serial.println(soilMoistureValue);

  String message = "{\"prompt\":\"Translate this temperature and humidity data into French: Temperature " + String(temperature) + "°C and Humidity " + String(humidity) + "%.\",\"max_tokens\":60}";
  String response = askGPT(message);
  Serial.println("Response: ");
  Serial.println(response);
}